# Analyze the effect of clustering on the speed data

<b>Intuition</b> : Classification is not possible to classify the speed range between 3.0mph-7.0mph as some people might be walking or running at intermittent speeds such as 4.0-6.0mph.

Therefore in this I have used different clustering algorithms. The challenge associated with this model is that we cannot test it against metrics such as homogeniety score or sihouette coeffecient. So a step in hyperparameter tuning involves using a RandomForestRegressor to rank the models based on the precision observed in the model.

## Algorithms used :
<ol>
    <li>KMeans</li>
    <li>DbScan</li>
    <li>Affinity Propagation</li>
</ol>

# Import Libraries

In [44]:
def rel(name):
    importlib.reload(sys.modules[name])
try:
    rel('DataBuilder')
    rel('MLData')
    rel('Heuristics')

except:
    print('importing for the first time')

from DataBuilder import DataBuilder
import ClassEvaluator
import numpy as np
import importlib,sys 
from IPython.display import clear_output

from sklearn.cluster import KMeans
from sklearn.metrics import *

import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split

    


In [7]:
import entropy


In [26]:
entropy


<module 'entropy' (namespace)>

# Build Data 

In [38]:
dataBuilder = DataBuilder(heuristic=False, applyFilter=False)

100%|██████████| 3/3 [00:36<00:00, 12.09s/it]


## KMeans

In [45]:
X_Train, y_Train, X_Test, y_Test = dataBuilder.confs['data1']['conf5']

#train_inds = np.where((y_Train<4.2)&(y_Train>6))
#test_inds = np.where((y_Test<4.2)&(y_Test>6))


#X_Train=X_Train[train_inds]
#y_Train=y_Train[train_inds]
#X_Test=X_Test[test_inds]
#y_Test=y_Test[test_inds]

In [46]:
#dataBuilder.confs


In [101]:
# Try kmeans for different number of clusters - 2  
def cluster_labels(arr, criteria):
    split_crirteria = [[[3,4],[5,7]],
                       [[3,4],[4,7]],
                       [[3,6],[6,7]],
                       [[3,4], [4,5] ,[5,7]],
                       [[3,3],[4,4],[5,5],[6,6],[7,7]]]


    label_no = 0
    split = split_crirteria[criteria]
    n_clusters = len(split)
    # Round all the clusters
    newArr = np.array([int(np.round(speed)) for speed in arr])
    # Modify labels of train and test according to the split criteria
    for subSplit in split:
        minim, maxim = subSplit
        indices = np.where((newArr==minim)|(newArr==maxim))
        newArr[indices] = label_no
        newArr = np.array([label_no if speed>=minim and speed<=maxim else speed  
                                 for speed in newArr])
        label_no+=1
        # fit and train the clustering algorithm with appropriate grid search criteria
    return(newArr.ravel())


In [102]:
y_test_conv = cluster_labels(y_Test, 0)
y_train_conv  = cluster_labels(y_Train, 0)

In [103]:
X_Train

array([[ -4.39130435,  -3.5       ,  -2.85869565, ..., 147.69896008,
        169.48165319, 196.16805597],
       [ 16.88043478,  11.85869565,  11.40217391, ..., 115.37964888,
        183.08828463, 211.55864949],
       [  7.58695652,   5.42391304,   5.63043478, ..., 256.52574521,
        293.37617864, 323.10728458],
       ...,
       [ -2.45652174,   0.32608696,  15.09782609, ..., 182.6494431 ,
        237.30991313, 239.8022703 ],
       [  1.65060241,   0.93975904,   0.4939759 , ...,  34.51200673,
         73.51943866, 117.40488689],
       [  3.52173913,   4.94565217,   6.07608696, ..., 155.51312383,
        162.95450654, 155.37864374]])

In [104]:
km = KMeans(n_clusters=2, max_iter=20000)
km.fit(X_Train)

KMeans(max_iter=20000, n_clusters=2)

In [105]:
adjusted_rand_score(y_train_conv.ravel(),km.labels_)
#accuracy_score(y_train_conv,km.labels_)

-0.00025808766811020637

In [106]:
np.hstack((y_train_conv.reshape(-1,1),km.labels_.reshape(-1,1)))

array([[1, 0],
       [1, 1],
       [1, 1],
       ...,
       [1, 0],
       [0, 0],
       [0, 1]])

In [107]:
import numpy as np
a=np.array([1,2,3,4,5,6,78])

In [108]:
np.where((a==1)|(a==2))

(array([0, 1]),)

In [109]:
inds = [0,1,2]

In [110]:
from sklearn.metrics import classification_report

print(classification_report(y_train_conv.reshape(-1,1),km.labels_.reshape(-1,1)))


              precision    recall  f1-score   support

           0       0.50      0.52      0.51      1823
           1       0.50      0.49      0.49      1810

    accuracy                           0.50      3633
   macro avg       0.50      0.50      0.50      3633
weighted avg       0.50      0.50      0.50      3633



In [114]:
print(confusion_matrix(y_train_conv.reshape(-1,1),km.labels_.reshape(-1,1)))

[[944 879]
 [930 880]]


In [115]:
# np.hstack((y_train_conv.reshape(-1,1),km.labels_.reshape(-1,1)))[20:40]


# Clustering Analysis on hand-crafted features

In [116]:
handDataBuilder = DataBuilder(heuristic=True, applyFilter=False)

100%|██████████| 1/1 [12:14<00:00, 734.11s/it]
